In [1]:
import pandas as pd
import arviz as az
import matplotlib.pyplot as plt
from scipy.stats import shapiro
from scipy.stats import skew


# There are two sets of results here, both are the conguate pair Bayesian CI's but the target for the first for modles is IV mean Skew and the target for the second four is HV

In [2]:
IV1 = pd.read_csv('IV_Known_Mean_Unknown_Var_[1, 1].csv')
IV2 = pd.read_csv('IV_Known_Mean_Unknown_Var_[2, 1].csv')
IV3 = pd.read_csv('IV_Known_Mean_Unknown_Var_[3, 0.5].csv')
IV4 = pd.read_csv('IV_Known_Mean_Unknown_Var_[3, 1].csv')
###
HV1 = pd.read_csv('HV_Known_Mean_Unknown_Var_[1, 1].csv')
HV2 = pd.read_csv('HV_Known_Mean_Unknown_Var_[2, 1].csv')
HV3 = pd.read_csv('HV_Known_Mean_Unknown_Var_[3, 0.5].csv')
HV4 = pd.read_csv('HV_Known_Mean_Unknown_Var_[3, 1].csv')
IV1.head()

,Unnamed: 0,Ticker,5_Day_CI,5_Day_EV,5_Day_RMSE,7_Day_CI,7_Day_EV,7_Day_RMSE,10_Day_CI,10_Day_EV,10_Day_RMSE,15_Day_CI,15_Day_EV,15_Day_RMSE,30_Day_CI,30_Day_EV,30_Day_RMSE
0,0,LUMN,0.979130,0.549565,0.124123,0.766957,0.547826,0.173451,0.121739,0.507826,0.241343,0.000000,0.507826,0.300553,0.0,0.507826,0.362343
1,1,VZ,0.998296,0.479841,0.230347,0.979557,0.482112,0.124795,0.964225,0.525270,0.067829,0.667235,0.547984,0.078895,0.0,0.519023,0.129377
2,2,T,0.994889,0.470187,0.213384,0.978421,0.487791,0.117191,0.913118,0.526405,0.083825,0.517320,0.546281,0.109401,0.0,0.529245,0.160633
3,3,DIS,0.990346,0.504259,0.193480,0.948325,0.499716,0.128255,0.736513,0.507098,0.133611,0.340716,0.510505,0.170595,0.0,0.499148,0.221918
4,4,TTWO,0.990914,0.503123,0.122019,0.848949,0.515048,0.136337,0.368541,0.509370,0.196208,0.000000,0.508234,0.253255,0.0,0.508234,0.314116


In [4]:
def add_sectors(dataframe):
    sectors = pd.read_csv('sectors.csv')
    final = dataframe.merge(sectors, on='Ticker',how='inner')
    final = final[['Ticker', '5_Day_CI', '5_Day_EV', '5_Day_RMSE','7_Day_CI', '7_Day_EV', '7_Day_RMSE', '10_Day_CI', '10_Day_EV','10_Day_RMSE', '15_Day_CI', '15_Day_EV', '15_Day_RMSE', '30_Day_CI','30_Day_EV', '30_Day_RMSE','Sector']]
    
    return(final)


def split_sectors(data,sectors):
    num_splits = len(sectors)
    dataframes = []
    
    for i in range(0,num_splits):
        dataframes.append(data[data['Sector'] == sectors[i]])
        
    final = pd.concat(dataframes,axis = 0)
    return(final)


In [5]:
IV1_sectors = add_sectors(IV1)
IV2_sectors = add_sectors(IV2)
IV3_sectors = add_sectors(IV3)
IV4_sectors = add_sectors(IV4)
###
HV1_sectors = add_sectors(HV1)
HV2_sectors = add_sectors(HV2)
HV3_sectors = add_sectors(HV3)
HV4_sectors = add_sectors(HV4)

IV1_sectors.head()

,Ticker,5_Day_CI,5_Day_EV,5_Day_RMSE,7_Day_CI,7_Day_EV,7_Day_RMSE,10_Day_CI,10_Day_EV,10_Day_RMSE,15_Day_CI,15_Day_EV,15_Day_RMSE,30_Day_CI,30_Day_EV,30_Day_RMSE,Sector
0,LUMN,0.979130,0.549565,0.124123,0.766957,0.547826,0.173451,0.121739,0.507826,0.241343,0.000000,0.507826,0.300553,0.0,0.507826,0.362343,Communication Services
1,VZ,0.998296,0.479841,0.230347,0.979557,0.482112,0.124795,0.964225,0.525270,0.067829,0.667235,0.547984,0.078895,0.0,0.519023,0.129377,Communication Services
2,T,0.994889,0.470187,0.213384,0.978421,0.487791,0.117191,0.913118,0.526405,0.083825,0.517320,0.546281,0.109401,0.0,0.529245,0.160633,Communication Services
3,DIS,0.990346,0.504259,0.193480,0.948325,0.499716,0.128255,0.736513,0.507098,0.133611,0.340716,0.510505,0.170595,0.0,0.499148,0.221918,Communication Services
4,TTWO,0.990914,0.503123,0.122019,0.848949,0.515048,0.136337,0.368541,0.509370,0.196208,0.000000,0.508234,0.253255,0.0,0.508234,0.314116,Communication Services


In [6]:
def summarize_results(data):
    
    list_of_results = []
    
    all_columns = data.describe()
    confidence_intervals = all_columns.iloc[:,::3]
    direction = all_columns.iloc[:,1::3]
    rmse = all_columns.iloc[:,2::3]
    
    list_of_results = [confidence_intervals,direction,rmse]
    
    
    return(list_of_results)
    
#split_sectors(data,['Communication Services', 'Consumer Discretionary', 'Consumer Staples'])

High_vol_sectors = ['Information Technology','Consumer Discretionary','Health Care','Energy','Communication Services']
Medium_vol_sectors = ['Materials','Industrials','Financials',]
Low_vol_sectors = ['Utilities','Real Estate','Consumer Staples']

In [44]:
a = summarize_results(HV4_sectors)

styled = a[2].iloc[0:]
styled = styled.style.background_gradient(axis = 1)
styled.set_table_attributes("style='display:inline'").set_caption('Alpha = 3, Beta = 1')
#styled.style.set_table_attributes("style='display:inline'").set_caption('Caption table')
styled
#dfi.export(styled,"mytable.png")#styled.export_png('styled.png')
#dfi.export(styled,'testing.png')
#a[1].head(10)
#a[2].head(10)

,5_Day_RMSE,7_Day_RMSE,10_Day_RMSE,15_Day_RMSE,30_Day_RMSE
count,487.000000,487.000000,487.000000,487.000000,487.000000
mean,0.210422,0.222229,0.240232,0.262523,0.294925
std,0.082489,0.088724,0.092715,0.094904,0.096034
min,0.109671,0.095664,0.097305,0.112475,0.143126
25%,0.154082,0.160576,0.176144,0.196374,0.229366
50%,0.184036,0.195518,0.213260,0.235582,0.268281
75%,0.238623,0.257569,0.280829,0.304848,0.339851
max,0.642380,0.647790,0.657293,0.690241,0.731664


In [55]:
a = summarize_results(split_sectors(IV1_sectors,High_vol_sectors))

styled = a[2].iloc[1:]
styled = styled.style.background_gradient(axis = 1)
styled
#styled = styled.style.background_gradient(axis = 1)
#styled

,5_Day_RMSE,7_Day_RMSE,10_Day_RMSE,15_Day_RMSE,30_Day_RMSE
mean,0.175977,0.154940,0.188190,0.236225,0.292950
std,0.045718,0.072920,0.090825,0.095787,0.096808
min,0.076913,0.050044,0.065394,0.076904,0.124223
25%,0.150364,0.108384,0.120740,0.166757,0.222140
50%,0.166714,0.126575,0.163087,0.212519,0.269079
75%,0.192606,0.171914,0.224197,0.278440,0.335083
max,0.419364,0.520106,0.599005,0.661965,0.725771


## Below was my first attempt, above is the same as the timeseries models

In [6]:
def analyze_results(names,sectors=['all']):
    
    #For each interval I will look at the 'Average_5_Day_CI_Accuracy','5_Day_CI_Range','5_Day_Prediction_Mean_Acc','5_Day_Prediction_Dist','5_Day_Avg_RMSE',
    
    results = pd.DataFrame(columns = ['Parameters','Average_5_Day_CI_Accuracy','5_Day_CI_Range','5_Day_Prediction_Mean_Acc','5_Day_Prediction_Dist','5_Day_Prediction_SKEW','5_Day_Avg_RMSE','Average_7_Day_CI_Accuracy','7_Day_CI_Range','7_Day_Prediction_Mean_Acc','7_Day_Prediction_Dist','7_Day_Prediction_SKEW','7_Day_Avg_RMSE','Average_10_Day_CI_Accuracy','10_Day_CI_Range','10_Day_Prediction_Mean_Acc','10_Day_Prediction_Dist','10_Day_Prediction_SKEW','10_Day_Avg_RMSE','Average_15_Day_CI_Accuracy','15_Day_CI_Range','15_Day_Prediction_Mean_Acc','15_Day_Prediction_Dist','15_Day_Prediction_SKEW','15_Day_Avg_RMSE','Average_30_Day_CI_Accuracy','30_Day_CI_Range','30_Day_Prediction_Mean_Acc','30_Day_Prediction_Dist','30_Day_Prediction_SKEW','30_Day_Avg_RMSE'])
    
    for name in names:
        
        
        row = []
        row.append(name)
        data_raw = pd.read_csv(f"{name}.csv")
        data = add_sectors(data_raw)
        
        if sectors[0] == 'all':
            
            
            for inc in ['5','7','10','15','30'] :
                
                # Average for the Credible Interval's Accuracy
                row.append(data[f'{inc}_Day_CI'].mean())
                # Range of Credible Interval's Accuracy
                row.append(((data[f'{inc}_Day_CI'].max() , data['5_Day_CI'].min())))
                # Prediction's Average Directional Accuracy accross all stocks
                row.append(data[f'{inc}_Day_EV'].mean())
                # Results of the Shapiro Test for Normality of Directional Accuracy ('is it centered around 50/50 ?')
                ### The null hypothesis is states that the population is normally distributed i.e if the p-value is greater than 0.05, then the null hypothesis is accepted
                row.append(shapiro(data[f'{inc}_Day_EV']))
                # Next is the skew of the Percentage Directional Accuracy
                row.append(skew(data[f'{inc}_Day_EV']))
                # Finally append the Average RMSE
                row.append(data[f'{inc}_Day_RMSE'].mean())
                
            results.loc[len(results.index)] = row
                   
        
        elif sectors[0] != 'all':
            
            
            data = split_sectors(data,sectors)
            
            for inc in ['5','7','10','15','30'] :
                
                # Average for the Credible Interval's Accuracy
                row.append(data[f'{inc}_Day_CI'].mean())
                # Range of Credible Interval's Accuracy
                row.append(((data[f'{inc}_Day_CI'].max() , data['5_Day_CI'].min())))
                # Prediction's Average Directional Accuracy accross all stocks
                row.append(data[f'{inc}_Day_EV'].mean())
                # Results of the Shapiro Test for Normality of Directional Accuracy ('is it centered around 50/50 ?')
                ### The null hypothesis is states that the population is normally distributed i.e if the p-value is greater than 0.05, then the null hypothesis is accepted
                row.append(shapiro(data[f'{inc}_Day_EV']))
                # Next is the skew of the Percentage Directional Accuracy
                row.append(skew(data[f'{inc}_Day_EV']))
                # Finally append the Average RMSE
                row.append(data[f'{inc}_Day_RMSE'].mean())
                
            results.loc[len(results.index)] = row
            
    return(results)
            
            
        
        
    

    
   
    
    
    

In [7]:
test = analyze_results(['Results_Experminent_One_[1, 1]_Trial_two','Results_Experminent_One_[2, 1]_Trial_two','Results_Experminent_One_[3, 0.5]_Trial_two','Results_Experminent_One_[3, 1]_Trial_two'],sectors = ['Information Technology','Health Care','Utilities'])
test.head() 

,Parameters,Average_5_Day_CI_Accuracy,5_Day_CI_Range,5_Day_Prediction_Mean_Acc,5_Day_Prediction_Dist,5_Day_Prediction_SKEW,5_Day_Avg_RMSE,Average_7_Day_CI_Accuracy,7_Day_CI_Range,7_Day_Prediction_Mean_Acc,...,15_Day_Prediction_Mean_Acc,15_Day_Prediction_Dist,15_Day_Prediction_SKEW,15_Day_Avg_RMSE,Average_30_Day_CI_Accuracy,30_Day_CI_Range,30_Day_Prediction_Mean_Acc,30_Day_Prediction_Dist,30_Day_Prediction_SKEW,30_Day_Avg_RMSE
0,"Results_Experminent_One_[1, 1]_Trial_two",0.984169,"(1.0, 0.5532544378698225)",0.509085,"(0.9069926738739014, 1.1675521882636986e-08)",1.553150,0.171484,0.902724,"(1.0, 0.5532544378698225)",0.521920,...,0.513782,"(0.9921035170555115, 0.5120686888694763)",-0.103676,0.200706,0.000112,"(0.0113571834185122, 0.5532544378698225)",0.508711,"(0.9754078984260559, 0.0052894530817866325)",-0.442547,0.257278
1,"Results_Experminent_One_[2, 1]_Trial_two",0.902435,"(1.0, 0.0029585798816568)",0.521947,"(0.9967067837715149, 0.9783869981765747)",-0.046085,0.132890,0.725212,"(0.9761363636363636, 0.0029585798816568)",0.527904,...,0.510327,"(0.9882122874259949, 0.18889792263507843)",-0.240067,0.213427,0.000077,"(0.0068143100511073, 0.0029585798816568)",0.508697,"(0.9757503271102905, 0.00579514866694808)",-0.441099,0.261025
2,"Results_Experminent_One_[3, 0.5]_Trial_two",0.122050,"(0.5820556501987507, 0.0)",0.509512,"(0.986962616443634, 0.13320159912109375)",-0.297744,0.219039,0.021345,"(0.2799545712663259, 0.0)",0.508787,...,0.508714,"(0.976204514503479, 0.006544675678014755)",-0.434927,0.269908,0.000032,"(0.0051502145922746, 0.0)",0.508690,"(0.9759076833724976, 0.006044086068868637)",-0.440140,0.291025
3,"Results_Experminent_One_[3, 1]_Trial_two",0.724862,"(0.9761363636363636, 0.0)",0.527935,"(0.9945275187492371, 0.8065215945243835)",-0.033036,0.145114,0.510127,"(0.9341283361726292, 0.0)",0.524781,...,0.509143,"(0.9855318069458008, 0.0887518897652626)",-0.325597,0.223799,0.000059,"(0.0051502145922746, 0.0)",0.508693,"(0.9758315086364746, 0.005922240670770407)",-0.440638,0.264343


In [8]:
test = analyze_results(['Results_Experminent_One_[1, 1]_Trial_two'],sectors = ['Information Technology','Health Care','Utilities'])
test.head()

,Parameters,Average_5_Day_CI_Accuracy,5_Day_CI_Range,5_Day_Prediction_Mean_Acc,5_Day_Prediction_Dist,5_Day_Prediction_SKEW,5_Day_Avg_RMSE,Average_7_Day_CI_Accuracy,7_Day_CI_Range,7_Day_Prediction_Mean_Acc,...,15_Day_Prediction_Mean_Acc,15_Day_Prediction_Dist,15_Day_Prediction_SKEW,15_Day_Avg_RMSE,Average_30_Day_CI_Accuracy,30_Day_CI_Range,30_Day_Prediction_Mean_Acc,30_Day_Prediction_Dist,30_Day_Prediction_SKEW,30_Day_Avg_RMSE
0,"Results_Experminent_One_[1, 1]_Trial_two",0.984169,"(1.0, 0.5532544378698225)",0.509085,"(0.9069926738739014, 1.1675521882636986e-08)",1.55315,0.171484,0.902724,"(1.0, 0.5532544378698225)",0.52192,...,0.513782,"(0.9921035170555115, 0.5120686888694763)",-0.103676,0.200706,0.000112,"(0.0113571834185122, 0.5532544378698225)",0.508711,"(0.9754078984260559, 0.0052894530817866325)",-0.442547,0.257278


In [52]:
#az.plot_kde(results['5_Day_CI'].values,rug=True)
